In [13]:
import selenium
from selenium import webdriver
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import signal
from tqdm import trange, notebook  
import time
import math

In [17]:
pip show selenium

Name: selenium
Version: 4.10.0
Summary: 
Home-page: https://www.selenium.dev
Author: 
Author-email: 
License: Apache 2.0
Location: c:\users\guddk\anaconda3\lib\site-packages
Requires: certifi, trio, trio-websocket, urllib3
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [22]:
driver = webdriver.Chrome()

In [24]:
# driver = webdriver.Chrome('./driver/chromedriver.exe')

In [23]:
url = 'https://www.wine21.com/13_search/wine_list.html'
driver.get(url)

In [25]:
# 평점순으로 (변경되기까지 시간이 오래걸림)
xpath = '//*[@id="shOrder1"]/option[4]'
driver.find_element(By.XPATH,xpath).click()

In [26]:
# 첫번째 와인 클릭 
driver.find_element(By.XPATH,'//*[@id="wine_list"]/li[1]/div[2]/div[1]/h3/a').click()
time.sleep(1)

In [69]:
result=[]

In [62]:
# 와인별 맛 데이터 
result = pd.DataFrame(columns=['당도','바디','산도','타닌'])

for i in range(1,5):
    elems = driver.find_elements(By.CSS_SELECTOR, 'body > section > div.inner > div.clear > div.wine-top-right > div.wine-components > ul > li:nth-child('+str(i)+') > div > a')
    a_on_count = 0

    for elem in elems :
        if elem.tag_name == 'a' and 'on' in elem.get_attribute('class') :
            a_on_count += 1
            
    # 해당 열에 값을 추가
    if i == 1:
        result.loc[0, '당도'] = a_on_count
    elif i == 2:
        result.loc[0, '바디'] = a_on_count
    elif i == 3:
        result.loc[0, '산도'] = a_on_count
    elif i == 4:
        result.loc[0, '타닌'] = a_on_count


In [63]:
result

,당도,바디,산도,타닌
0,1,3,3,3


In [80]:
# 와인별 맛 데이터 
result = pd.DataFrame(columns=['당도','바디','산도','타닌'])
for i in notebook.tqdm(range(1,401)):
    # 와인 클릭 
    driver.find_element(By.CSS_SELECTOR,'#wine_list > li:nth-child('+str(i)+') > div.txt-area > div.cnt-header > h3 > a').send_keys(Keys.ENTER)
    time.sleep(4)

    # 새로운 프레임 생성
    new_result = pd.DataFrame(columns=['당도','바디','산도','타닌'])
    for a in range(1,5):
        elems = driver.find_elements(By.CSS_SELECTOR, 'body > section > div.inner > div.clear > div.wine-top-right > div.wine-components > ul > li:nth-child('+str(a)+') > div > a')
        a_on_count = 0

        for elem in elems :
            if elem.tag_name == 'a' and 'on' in elem.get_attribute('class') :
                a_on_count += 1

        # 해당 열에 값을 추가
        if a == 1:
            result.loc[0, '당도'] = a_on_count
        elif a == 2:
            result.loc[0, '바디'] = a_on_count
        elif a == 3:
            result.loc[0, '산도'] = a_on_count
        elif a == 4:
            result.loc[0, '타닌'] = a_on_count

    result = pd.concat([result, new_result], ignore_index=True)
    time.sleep(3)
    # 전체 목록으로 돌아가기 
    driver.back()    
    time.sleep(8)


  0%|          | 0/200 [00:00<?, ?it/s]

,당도,바디,산도,타닌
0,1,4,5,4


In [24]:
result=[]
result = pd.DataFrame()
for i in notebook.tqdm(range(0,401)):
    try : 
        # 와인 클릭 
        driver.find_element(By.XPATH,'//*[@id="wine_list"]/li['+str(i)+']/div[2]/div[1]/h3/a').click()
        time.sleep(3)
        
        # 가져올 순서 -> 중분류, 상품명, 생산국가, 상세정보
       
        wine_category=driver.find_element(By.XPATH, '/html/body/section/div[3]/div[2]/div[2]/div[1]/p/span[1]').text
        wine_name = driver.find_element(By.CLASS_NAME,'wine-name').text
        wine_origin = driver.find_element(By.XPATH, '/html/body/section/div[3]/div[2]/div[2]/div[1]/p/span[2]').text

        wine_detail = driver.find_element(By.XPATH, '//*[@id="detail"]/div/div').text
        
        # 전체 목록으로 돌아가기 
        driver.back()        
        time.sleep(8)
        
        # 데이터프레임에 저장
        result = result.append(pd.DataFrame({'중분류':wine_category, '상품명':wine_name,'생산국가':wine_origin,'상세정보':wine_detail},index=['wine'])) 

    except :
        pass

  0%|          | 0/401 [00:00<?, ?it/s]

C:\Users\TECH4-05\AppData\Local\Temp\ipykernel_7164\3126024163.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(pd.DataFrame({'중분류':wine_category, '상품명':wine_name,'생산국가':wine_origin,'상세정보':wine_detail},index=['wine']))
C:\Users\TECH4-05\AppData\Local\Temp\ipykernel_7164\3126024163.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(pd.DataFrame({'중분류':wine_category, '상품명':wine_name,'생산국가':wine_origin,'상세정보':wine_detail},index=['wine']))
C:\Users\TECH4-05\AppData\Local\Temp\ipykernel_7164\3126024163.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(pd.DataFrame({'중분류':wine_category, '상품명':wine_name,'생산국가':wine_origin,'상세정보':wine_detail},index=['wine

C:\Users\TECH4-05\AppData\Local\Temp\ipykernel_7164\3126024163.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(pd.DataFrame({'중분류':wine_category, '상품명':wine_name,'생산국가':wine_origin,'상세정보':wine_detail},index=['wine']))
C:\Users\TECH4-05\AppData\Local\Temp\ipykernel_7164\3126024163.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(pd.DataFrame({'중분류':wine_category, '상품명':wine_name,'생산국가':wine_origin,'상세정보':wine_detail},index=['wine']))
C:\Users\TECH4-05\AppData\Local\Temp\ipykernel_7164\3126024163.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(pd.DataFrame({'중분류':wine_category, '상품명':wine_name,'생산국가':wine_origin,'상세정보':wine_detail},index=['wine

C:\Users\TECH4-05\AppData\Local\Temp\ipykernel_7164\3126024163.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(pd.DataFrame({'중분류':wine_category, '상품명':wine_name,'생산국가':wine_origin,'상세정보':wine_detail},index=['wine']))
C:\Users\TECH4-05\AppData\Local\Temp\ipykernel_7164\3126024163.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(pd.DataFrame({'중분류':wine_category, '상품명':wine_name,'생산국가':wine_origin,'상세정보':wine_detail},index=['wine']))
C:\Users\TECH4-05\AppData\Local\Temp\ipykernel_7164\3126024163.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(pd.DataFrame({'중분류':wine_category, '상품명':wine_name,'생산국가':wine_origin,'상세정보':wine_detail},index=['wine

C:\Users\TECH4-05\AppData\Local\Temp\ipykernel_7164\3126024163.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(pd.DataFrame({'중분류':wine_category, '상품명':wine_name,'생산국가':wine_origin,'상세정보':wine_detail},index=['wine']))
C:\Users\TECH4-05\AppData\Local\Temp\ipykernel_7164\3126024163.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(pd.DataFrame({'중분류':wine_category, '상품명':wine_name,'생산국가':wine_origin,'상세정보':wine_detail},index=['wine']))
C:\Users\TECH4-05\AppData\Local\Temp\ipykernel_7164\3126024163.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(pd.DataFrame({'중분류':wine_category, '상품명':wine_name,'생산국가':wine_origin,'상세정보':wine_detail},index=['wine

...

C:\Users\TECH4-05\AppData\Local\Temp\ipykernel_7164\3126024163.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(pd.DataFrame({'중분류':wine_category, '상품명':wine_name,'생산국가':wine_origin,'상세정보':wine_detail},index=['wine']))
C:\Users\TECH4-05\AppData\Local\Temp\ipykernel_7164\3126024163.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(pd.DataFrame({'중분류':wine_category, '상품명':wine_name,'생산국가':wine_origin,'상세정보':wine_detail},index=['wine']))
C:\Users\TECH4-05\AppData\Local\Temp\ipykernel_7164\3126024163.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(pd.DataFrame({'중분류':wine_category, '상품명':wine_name,'생산국가':wine_origin,'상세정보':wine_detail},index=['wine

C:\Users\TECH4-05\AppData\Local\Temp\ipykernel_7164\3126024163.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(pd.DataFrame({'중분류':wine_category, '상품명':wine_name,'생산국가':wine_origin,'상세정보':wine_detail},index=['wine']))
C:\Users\TECH4-05\AppData\Local\Temp\ipykernel_7164\3126024163.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(pd.DataFrame({'중분류':wine_category, '상품명':wine_name,'생산국가':wine_origin,'상세정보':wine_detail},index=['wine']))
C:\Users\TECH4-05\AppData\Local\Temp\ipykernel_7164\3126024163.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(pd.DataFrame({'중분류':wine_category, '상품명':wine_name,'생산국가':wine_origin,'상세정보':wine_detail},index=['wine

C:\Users\TECH4-05\AppData\Local\Temp\ipykernel_7164\3126024163.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(pd.DataFrame({'중분류':wine_category, '상품명':wine_name,'생산국가':wine_origin,'상세정보':wine_detail},index=['wine']))
C:\Users\TECH4-05\AppData\Local\Temp\ipykernel_7164\3126024163.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(pd.DataFrame({'중분류':wine_category, '상품명':wine_name,'생산국가':wine_origin,'상세정보':wine_detail},index=['wine']))
C:\Users\TECH4-05\AppData\Local\Temp\ipykernel_7164\3126024163.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(pd.DataFrame({'중분류':wine_category, '상품명':wine_name,'생산국가':wine_origin,'상세정보':wine_detail},index=['wine

C:\Users\TECH4-05\AppData\Local\Temp\ipykernel_7164\3126024163.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(pd.DataFrame({'중분류':wine_category, '상품명':wine_name,'생산국가':wine_origin,'상세정보':wine_detail},index=['wine']))
C:\Users\TECH4-05\AppData\Local\Temp\ipykernel_7164\3126024163.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(pd.DataFrame({'중분류':wine_category, '상품명':wine_name,'생산국가':wine_origin,'상세정보':wine_detail},index=['wine']))
C:\Users\TECH4-05\AppData\Local\Temp\ipykernel_7164\3126024163.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(pd.DataFrame({'중분류':wine_category, '상품명':wine_name,'생산국가':wine_origin,'상세정보':wine_detail},index=['wine

C:\Users\TECH4-05\AppData\Local\Temp\ipykernel_7164\3126024163.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(pd.DataFrame({'중분류':wine_category, '상품명':wine_name,'생산국가':wine_origin,'상세정보':wine_detail},index=['wine']))
C:\Users\TECH4-05\AppData\Local\Temp\ipykernel_7164\3126024163.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(pd.DataFrame({'중분류':wine_category, '상품명':wine_name,'생산국가':wine_origin,'상세정보':wine_detail},index=['wine']))
C:\Users\TECH4-05\AppData\Local\Temp\ipykernel_7164\3126024163.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(pd.DataFrame({'중분류':wine_category, '상품명':wine_name,'생산국가':wine_origin,'상세정보':wine_detail},index=['wine

In [25]:
result 

,중분류,상품명,생산국가,상세정보
wine,레드,"GCF, 꼬뜨 드 뵈프",프랑스,생산자\nGCF 그룹 GCF Group\n국가/생산지역\n프랑스(France) > ...
wine,화이트,"가트, 리슬링",호주,생산자\n가트 와인즈 Gatt Wines\n국가/생산지역\n호주(Australia)...
wine,레드,갈로 리빙스톤 콩코드,미국,생산자\n갤로 패밀리 빈야드 Gallo Family Vineyard - E&J Ga...
wine,주정강화,글로리아 토니 포트 와인,포르투갈,생산자\n빈센테 파리아 비뉴스 Vincente Faria VInhos\n국가/생산지...
wine,레드,"기욤 르탕, 루비 루즈",프랑스,생산자\n기욤 르탕 Guillaume Letang\n국가/생산지역\n프랑스(Fran...
...,...,...,...,...
wine,레드,"매티스, 그르나슈",미국,생산자\n매티스 와이너리 Mathis Winery\n국가/생산지역\n미국(U.S.A...
wine,화이트,"메종 카스텔, 소비뇽 블랑",프랑스,생산자\n메종 카스텔 Maison Castel\n국가/생산지역\n프랑스(France...
wine,스파클링,"메종 카스텔, 퀴베 블랑쉬 브뤼",프랑스,생산자\n메종 카스텔 Maison Castel\n국가/생산지역\n프랑스(France...
wine,화이트,"메종 카스텔, 푸이 퓌메",프랑스,생산자\n메종 카스텔 Maison Castel\n국가/생산지역\n프랑스(France...


In [26]:
# csv 파일로 저장하기 
result.to_csv('./wine400_list.csv', encoding = 'utf-8-sig')